In [ ]:
import os
import tensorflow as tf

# importing the images from the zip file
from google.colab import drive
drive.mount('/content/drive') 

!unzip /content/drive/MyDrive/DeepLearning/chest_xray2.zip

In [ ]:
train_directory = '/content/chest_xray/train'
validation_directory = '/content/chest_xray/validation'
test_directory = '/content/chest_xray/test'
BatchSize=64

TrainData = tf.keras.utils.image_dataset_from_directory(
    train_directory, seed=3, validation_split=None, subset=None, batch_size=BatchSize, shuffle=True,image_size=(256, 256),label_mode='categorical')
    
ValidationData = tf.keras.utils.image_dataset_from_directory(
    validation_directory, seed=3, validation_split=None, subset=None,shuffle=False, batch_size=BatchSize,image_size=(256, 256),label_mode='categorical')

TestData = tf.keras.utils.image_dataset_from_directory(
    test_directory, seed=3, validation_split=None,shuffle=False, subset=None, batch_size=879,image_size=(256, 256),label_mode='categorical')

In [ ]:
from numpy.core.fromnumeric import shape
import numpy as np
import tensorflow as tf
numEpochs = 8

model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(256,256,3)),
        tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Conv2D(128, kernel_size=(3, 3), activation="relu"),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(8),
        tf.keras.layers.Dense(2, activation="softmax"),
])

model.compile(optimizer="Adam",
              loss="categorical_crossentropy", 
              metrics=['accuracy'])

model.summary()

In [ ]:
history = model.fit(TrainData, epochs=numEpochs, validation_data=ValidationData)

In [ ]:
import matplotlib.pyplot as plt

ax = plt.subplot(1,1,1)
line1 = ax.plot(range(1,numEpochs+1),history.history['loss'])
line2 = ax.plot(range(1,numEpochs+1),history.history['val_loss'])
plt.ylim(0,0.5)
plt.legend(["train loss", "val loss"])
plt.grid(visible=True, which='both', axis='both')
plt.show()

In [ ]:
ax = plt.subplot(1,1,1)
line1 = ax.plot(range(1,numEpochs+1),history.history['accuracy'])
line2 = ax.plot(range(1,numEpochs+1),history.history['val_accuracy'])
plt.legend(["accuracy", "val_accuracy"])
plt.grid(visible=True, which='both', axis='both')
plt.show()

In [ ]:
loss, accuracy = model.evaluate(TestData)
print("Loss :", loss)
print("Accuracy :", accuracy)

In [ ]:
#confusion matrix

predictions = model.predict(TestData)
predictions = np.argmin(predictions, axis=1)

test_y = next(TestData.as_numpy_iterator())[1]
test_y = np.argmin(test_y, axis = 1)

confusion_mat = tf.math.confusion_matrix(test_y,predictions)
TP = confusion_mat.numpy()[0,0];
FP = confusion_mat.numpy()[1,0];
FN = confusion_mat.numpy()[0,1];
TN = confusion_mat.numpy()[1,1];

precision=TP / (TP + FP)
recall=TP / (TP + FN)
print("Precision: " + str(precision));
print("Recall: " + str(recall));
print("confusion matrix: " + str(confusion_mat))